In [1]:
import requests
import pandas as pd
from datetime import date
from io import BytesIO

from datetime import datetime
from dateutil.parser import parse
from tqdm import tqdm

import pymongo
from pymongo import MongoClient
from pymongo.server_api import ServerApi

In [2]:
# class to determine ganjil genap
def is_date(string, fuzzy=False):
    try: 
        parse(string, fuzzy=fuzzy)
        return True
    except ValueError:
        return False

In [3]:
# reading all BL number from google sheet
from acquiringbl import takingBL
bl_list = takingBL("KMTC")

In [4]:
# array for failed tracking
gagal=[]

In [5]:
hasil_akhir = []
for i, bls in enumerate(tqdm(bl_list)):
    try:
        # hitting the API
        url_ctr = "https://api.ekmtc.com/trans/trans/cargo-tracking/{}?dtKnd=BL&blNo={}".format(
            bls[4:], bls[4:])
        response_ctr = requests.request("GET", url_ctr)

        # getting container data. booking id needed for API URL
        ctr_data = response_ctr.json()
        list_of_container_number = []
        list_of_bookingid = []

        for j, ctr_info in enumerate(ctr_data['cntrList']):
            list_of_container_number.append(ctr_info["cntrNo"])
            list_of_bookingid.append(ctr_info["bkgNo"])

        for c, ctr in enumerate(list_of_container_number):
            url_milestone = "https://api.ekmtc.com/trans/trans/cargo-tracking/{}/detail?bkgNo={}&cntrNo={}&dtKnd=BL&strBkgNo={}".format(
                bls[4:], list_of_bookingid[c], ctr, list_of_bookingid[c])
            
            # hitting the API again for every container for container milestones
            response_milestone = requests.request("GET", url_milestone)

            # aqcuiring milestone datas for every container
            milstone_data = response_milestone.json()
            current_dict = {}
            dict_milestone = {}
            milestones = []
            key_mapping = {
                'blNo': 'BL Number',
                'cntrNo': 'Container Number',
                'etd': 'ETD',
                'eta': 'ETA',
                # 'GTOOB':'Gate Out Origin',
                # 'GTIOB':'Gate In Origin',
                'LDGOB': 'ATD',
                'DISIB': 'ATA',
                'GTOIB': 'Container Release',
                'GTIIB': 'Container Return'
            }

            current_dict.update({"Liners": "KMTC"})
            current_dict.update(
                {"From": milstone_data['trackingList'][-1]['plcNm'][:milstone_data['trackingList'][-1]['plcNm'].index(",")]})
            current_dict.update(
                {"To": milstone_data['trackingList'][-1]['podPortNm']})

            for key, label in key_mapping.items():
                if key in ctr_data['cntrList'][c]:
                    if key == "etd" or key == "eta":
                        current_dict[label] = datetime.strptime(
                            ctr_data['cntrList'][c][key][:8], "%Y%m%d").strftime("%Y-%m-%d")
                    else:
                        current_dict[label] = ctr_data['cntrList'][c][key]

            for b, milestone in enumerate(milstone_data['trackingList']):
                milestones.append(
                    milestone["cntrStsCd"] + milestone["cntrMvntCd"])
                milestones.append(milestone["mvntDt"])

            for m, milestone in enumerate(milestones):
                if is_date(milestone):
                    case_milestone = {
                        milestones[m-1]: datetime.strptime(milestone, "%Y%m%d").strftime("%Y-%m-%d")}
                    dict_milestone.update(case_milestone)

            for key, label in key_mapping.items():
                if key in dict_milestone:
                    current_dict[label] = dict_milestone[key]

            print(current_dict)
            hasil_akhir.append(current_dict)
    except Exception as e:
        print(e)
        print("{} GAGAL!!".format(bls))
        gagal.append(bls)


  7%|▋         | 1/15 [00:02<00:36,  2.62s/it]

{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'NORTH MANILA', 'BL Number': 'JKT4435445', 'Container Number': 'FFAU3967342', 'ETD': '2023-04-07', 'ETA': '2023-04-14', 'ATD': '2023-04-07', 'ATA': '2023-04-14', 'Container Release': '2023-05-12'}


 13%|█▎        | 2/15 [00:03<00:17,  1.38s/it]

{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'NORTH MANILA', 'BL Number': 'JKT4479884', 'Container Number': 'BMOU6345887', 'ETD': '2023-06-02', 'ETA': '2023-06-09', 'ATD': '2023-06-02', 'ATA': '2023-06-09'}
{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'HOCHIMINH', 'BL Number': 'JKT4589520', 'Container Number': 'FCIU7472518', 'ETD': '2023-09-14', 'ETA': '2023-09-20', 'ATD': '2023-09-14', 'ATA': '2023-09-20'}


 20%|██        | 3/15 [00:04<00:15,  1.29s/it]

...

{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'HOCHIMINH', 'BL Number': 'JKT4587697', 'Container Number': 'SEGU6586346', 'ETD': '2023-09-14', 'ETA': '2023-09-20', 'ATD': '2023-09-14', 'ATA': '2023-09-20'}
{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'HOCHIMINH', 'BL Number': 'JKT4581147', 'Container Number': 'FCIU7270517', 'ETD': '2023-09-10', 'ETA': '2023-09-13', 'ATD': '2023-09-10', 'ATA': '2023-09-13'}
{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'HOCHIMINH', 'BL Number': 'JKT4581147', 'Container Number': 'FFAU1044834', 'ETD': '2023-09-10', 'ETA': '2023-09-13', 'ATD': '2023-09-10', 'ATA': '2023-09-13'}
{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'HOCHIMINH', 'BL Number': 'JKT4581147', 'Container Number': 'FFAU1058376', 'ETD': '2023-09-10', 'ETA': '2023-09-13', 'ATD': '2023-09-10', 'ATA': '2023-09-13'}
{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'HOCHIMINH', 'BL Number': 'JKT4581147', 'Container Number': 'FFAU1064147', 'ETD': '2023-09-10', 'ETA': '2023-09-13', 'ATD': '2023-09-10', 'ATA':

 93%|█████████▎| 14/15 [00:33<00:02,  2.38s/it]

{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'HOCHIMINH', 'BL Number': 'JKT4581147', 'Container Number': 'KMTU9361194', 'ETD': '2023-09-10', 'ETA': '2023-09-13', 'ATD': '2023-09-10', 'ATA': '2023-09-13'}
{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'SHEKOU', 'BL Number': 'JKT4599082', 'Container Number': 'BEAU4870270', 'ETD': '2023-09-19', 'ETA': '2023-09-26', 'ATD': '2023-09-19', 'ATA': '2023-09-26', 'Container Release': '2023-09-27', 'Container Return': '2023-09-27'}
{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'SHEKOU', 'BL Number': 'JKT4599082', 'Container Number': 'BMOU6809231', 'ETD': '2023-09-19', 'ETA': '2023-09-26', 'ATD': '2023-09-19', 'ATA': '2023-09-26'}
{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'SHEKOU', 'BL Number': 'JKT4599082', 'Container Number': 'BMOU6817432', 'ETD': '2023-09-19', 'ETA': '2023-09-26', 'ATD': '2023-09-19', 'ATA': '2023-09-26'}
{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'SHEKOU', 'BL Number': 'JKT4599082', 'Container Number': 'BMOU6926639', 'ETD': '2023

100%|██████████| 15/15 [00:39<00:00,  2.65s/it]

{'Liners': 'KMTC', 'From': 'JAKARTA', 'To': 'SHEKOU', 'BL Number': 'JKT4599082', 'Container Number': 'UETU5452622', 'ETD': '2023-09-19', 'ETA': '2023-09-26', 'ATD': '2023-09-19', 'ATA': '2023-09-26'}


In [6]:
# connect to mongodb
from mongoinit import mongo_table_initiation, insert_many_mongo

mongo_table_initiation()
insert_many_mongo(hasil_akhir)

Today's Collection Name ===>  all_tracking_Oct-02-2023
Today's Collection Has Been Made
Inserting Many Complete!!


In [7]:
gagal

[]

In [7]:
# exporting to excel file
df = pd.DataFrame(hasil_akhir)
excel_file_path = 'export excel/KMTC.xlsx'
df.to_excel(excel_file_path, index=False)